In [146]:
!pip install nltk

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [147]:
import numpy as np 
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from sklearn.metrics import classification_report

In [148]:
data = pd.read_csv('dataset/train_preprocessed.csv')
data.dropna(inplace = True)

In [149]:
MAX_NB_WORDS = 11000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))



Found 6612 unique tokens.


In [150]:
word_index

{'jersey': 3492,
 'delete': 186,
 'primarily': 2477,
 'clear': 423,
 'referenced': 3884,
 '2015': 1162,
 'watercraft': 5518,
 'inquiries': 1716,
 'chosen': 3551,
 'middle': 5915,
 'administration': 494,
 'poster': 6165,
 'please': 36,
 'matchmaking': 4730,
 '486': 4576,
 'describe': 1560,
 'kongregate': 5255,
 'explaining': 2791,
 'traveling': 4557,
 "browser's": 1101,
 '85254': 5371,
 "fsd's": 4884,
 'controller': 1337,
 'images': 1772,
 '2005': 2639,
 '300': 5758,
 "merchant's": 3227,
 'civictour': 3866,
 'increased': 4807,
 'exchange': 1057,
 '0564': 5990,
 'ten': 687,
 '6401': 4133,
 'branding': 4244,
 'usage': 157,
 'tag': 668,
 'configure': 2250,
 'instant': 1290,
 'positive': 2699,
 'degree': 5561,
 'maintains': 1525,
 'electronic': 303,
 'buffa': 5105,
 'newspapers': 4533,
 'faculty': 3519,
 'noticeto': 5905,
 'theunited': 5218,
 'virtue': 2254,
 'remains': 1975,
 'ommunicate': 6458,
 'archival': 2000,
 'allaboutcookies': 2612,
 'application': 104,
 'hudson': 6516,
 'sellers': 

In [151]:
X = tokenizer.texts_to_sequences(data['text'].values)
print(len(X[20]))
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

35
Shape of data tensor: (19357, 100)


In [152]:
Y = pd.get_dummies(data['class']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (19357, 10)


In [153]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42, stratify=Y)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(15485, 100) (15485, 10)
(3872, 100) (3872, 10)


In [154]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
print(model.summary())

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 100, 100)          1100000   
_________________________________________________________________
spatial_dropout1d_16 (Spatia (None, 100, 100)          0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_29 (Dense)             (None, 10)                1010      
Total params: 1,181,410
Trainable params: 1,181,410
Non-trainable params: 0
_________________________________________________________________
None


In [155]:
epochs = 5
batch_size = 64

history = model.fit(
    X_train, Y_train, 
    epochs=epochs, batch_size=batch_size,
    validation_split=0.2,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]
)

d:\projects\python projects\venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 12388 samples, validate on 3097 samples
Epoch 1/5
12388/12388 [==============================] - 57s 5ms/step - loss: 1.4820 - categorical_accuracy: 0.4999 - val_loss: 1.0226 - val_categorical_accuracy: 0.6677
Epoch 2/5
12388/12388 [==============================] - 59s 5ms/step - loss: 0.8965 - categorical_accuracy: 0.7155 - val_loss: 0.7794 - val_categorical_accuracy: 0.7627
Epoch 3/5
12388/12388 [==============================] - 59s 5ms/step - loss: 0.6442 - categorical_accuracy: 0.7950 - val_loss: 0.6914 - val_categorical_accuracy: 0.7904
Epoch 4/5
12388/12388 [==============================] - 60s 5ms/step - loss: 0.5238 - categorical_accuracy: 0.8314 - val_loss: 0.6500 - val_categorical_accuracy: 0.7982
Epoch 5/5
12388/12388 [==============================] - 61s 5ms/step - loss: 0.4442 - categorical_accuracy: 0.8661 - val_loss: 0.6661 - val_categorical_accuracy: 0.8079


In [156]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

3872/3872 [==============================] - 3s 886us/step
Test set
  Loss: 0.666
  Accuracy: 0.803
